   Copyright 2020 Erik Lee

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

In [1]:
import os
import zipfile
import random
import shutil
from shutil import copyfile
import cv2.cv2 as cv2

import tensorflow as tf
import numpy as np
import matplotlib as plt
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from stn import spatial_transformer_network as transformer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas

i_s = 80
tf.keras.backend.clear_session()
%precision 4

L2_WEIGHT_DECAY = 0.01
L1_WEIGHT_DECAY = 0.003

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
# add bias regularlizers??

input_layer = tf.keras.layers.Input(shape=(i_s, i_s, 3))
# x = tf.keras.layers.Conv2D(64, (3,3), kernel_initializer='he_normal')(input_layer)
# x = tf.keras.layers.Conv2D(64, (3,3), kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(L2_WEIGHT_DECAY))(input_layer)
x = tf.keras.layers.Conv2D(64, (3,3), kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(input_layer)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation("relu")(x)

# x = tf.keras.layers.Conv2D(64, (3,3), kernel_initializer='he_normal')(x)
x = tf.keras.layers.Conv2D(64, (3,3), kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation("relu")(x)

x = tf.keras.layers.MaxPool2D((2,2), strides=2)(x)
x = tf.keras.layers.BatchNormalization()(x)

# p1a = tf.keras.layers.Conv2D(128, (1,1), padding="same", kernel_initializer='he_normal')(x)
p1a = tf.keras.layers.Conv2D(128, (1,1), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
p1a = tf.keras.layers.BatchNormalization()(p1a)
p1a = tf.keras.layers.Activation("relu")(p1a)

# p1a = tf.keras.layers.Conv2D(128, (3,3), padding="same", kernel_initializer='he_normal')(p1a)
p1a = tf.keras.layers.Conv2D(128, (3,3), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(p1a)
p1a = tf.keras.layers.BatchNormalization()(p1a)
p1a = tf.keras.layers.Activation("relu")(p1a)

p1b = tf.keras.layers.MaxPool2D((3,3), padding="same", strides=1)(x)

# p1b = tf.keras.layers.Conv2D(128, (1,1), padding="same", kernel_initializer='he_normal')(p1b)
p1b = tf.keras.layers.Conv2D(128, (1,1), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(p1b)
p1b = tf.keras.layers.BatchNormalization()(p1b)
p1b = tf.keras.layers.Activation("relu")(p1b)

# p1b = tf.keras.layers.Conv2D(128, (3,3), padding="same", kernel_initializer='he_normal')(p1b)
p1b = tf.keras.layers.Conv2D(128, (3,3), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(p1b)
p1b = tf.keras.layers.BatchNormalization()(p1b)
p1b = tf.keras.layers.Activation("relu")(p1b)

cat_256 = tf.keras.layers.Concatenate(axis=-1)([p1a, p1b])

x = tf.keras.layers.MaxPool2D((2,2), strides=2)(cat_256)

# p2a = tf.keras.layers.Conv2D(256, (1,1), padding="same", kernel_initializer='he_normal')(x)
p2a = tf.keras.layers.Conv2D(256, (1,1), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
p2a = tf.keras.layers.BatchNormalization()(p2a)
p2a = tf.keras.layers.Activation("relu")(p2a)

# p2a = tf.keras.layers.Conv2D(256, (3,3), padding="same", kernel_initializer='he_normal')(p2a)
p2a = tf.keras.layers.Conv2D(256, (3,3), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(p2a)
p2a = tf.keras.layers.BatchNormalization()(p2a)
p2a = tf.keras.layers.Activation("relu")(p2a)

p2b = tf.keras.layers.MaxPool2D((3,3), strides=1, padding="same")(x)

# p2b = tf.keras.layers.Conv2D(256, (1,1), padding="same", kernel_initializer='he_normal')(p2b)
p2b = tf.keras.layers.Conv2D(256, (1,1), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(p2b)
p2b = tf.keras.layers.BatchNormalization()(p2b)
p2b = tf.keras.layers.Activation("relu")(p2b)

# p2b = tf.keras.layers.Conv2D(256, (3,3), padding="same", kernel_initializer='he_normal')(p2b)
p2b = tf.keras.layers.Conv2D(256, (3,3), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal" kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(p2b)
p2b = tf.keras.layers.BatchNormalization()(p2b)
p2b = tf.keras.layers.Activation("relu")(p2b)

cat_512 = tf.keras.layers.Concatenate()([p2a, p2b])

x = tf.keras.layers.MaxPool2D((2,2), strides=2)(cat_512)

# x = tf.keras.layers.Conv2D(512, (1,1),  padding="same", kernel_initializer='he_normal')(x)
x = tf.keras.layers.Conv2D(512, (1,1),  padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation("relu")(x)

x = tf.keras.layers.MaxPool2D((2,2), strides=2)(x)

# x = tf.keras.layers.Conv2D(512, (3,3), kernel_initializer='he_normal')(x)
x = tf.keras.layers.Conv2D(512, (3,3), kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation("relu")(x)

x = tf.keras.layers.MaxPool2D((2,2), strides=2)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Flatten()(x)

# x = tf.keras.layers.Dense(1024, activation="relu", kernel_initializer='he_normal')(x)
x = tf.keras.layers.Dense(1024, activation="relu", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation="relu", kernel_initializer='he_normal', bias_initializer="he_normal")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(5, activation="softmax", kernel_initializer='he_normal', bias_initializer="he_normal")(x)

model = Model(input_layer, x, name='xNet_reg')
model.summary()

Model: "xNet_reg"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 78, 78, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 78, 78, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 78, 78, 64)   0           batch_normalization[0][0]        
___________________________________________________________________________________________

In [4]:
# input_layer = tf.keras.layers.Input(shape=(i_s, i_s, 3))
# x = tf.keras.layers.Conv2D(64, (3,3), kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(input_layer)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Activation("relu")(x)

# x = tf.keras.layers.Conv2D(64, (3,3), kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Activation("relu")(x)

# x = tf.keras.layers.MaxPool2D((2,2), strides=2)(x)
# x = tf.keras.layers.BatchNormalization()(x)

# p1a = tf.keras.layers.Conv2D(128, (1,1), padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(x)
# p1a = tf.keras.layers.BatchNormalization()(p1a)
# p1a = tf.keras.layers.Activation("relu")(p1a)

# p1a = tf.keras.layers.Conv2D(128, (3,3), padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(p1a)
# p1a = tf.keras.layers.BatchNormalization()(p1a)
# p1a = tf.keras.layers.Activation("relu")(p1a)

# p1b = tf.keras.layers.MaxPool2D((3,3), padding="same", strides=1)(x)

# p1b = tf.keras.layers.Conv2D(128, (1,1), padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(p1b)
# p1b = tf.keras.layers.BatchNormalization()(p1b)
# p1b = tf.keras.layers.Activation("relu")(p1b)

# p1b = tf.keras.layers.Conv2D(128, (3,3), padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(p1b)
# p1b = tf.keras.layers.BatchNormalization()(p1b)
# p1b = tf.keras.layers.Activation("relu")(p1b)

# cat_256 = tf.keras.layers.Concatenate(axis=-1)([p1a, p1b])

# x = tf.keras.layers.MaxPool2D((2,2), strides=2)(cat_256)

# p2a = tf.keras.layers.Conv2D(256, (1,1), padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(x)
# p2a = tf.keras.layers.BatchNormalization()(p2a)
# p2a = tf.keras.layers.Activation("relu")(p2a)

# p2a = tf.keras.layers.Conv2D(256, (3,3), padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(p2a)
# p2a = tf.keras.layers.BatchNormalization()(p2a)
# p2a = tf.keras.layers.Activation("relu")(p2a)

# p2b = tf.keras.layers.MaxPool2D((3,3), strides=1, padding="same")(x)

# p2b = tf.keras.layers.Conv2D(256, (1,1), padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(p2b)
# p2a = tf.keras.layers.BatchNormalization()(p2b)
# p2a = tf.keras.layers.Activation("relu")(p2b)

# p2b = tf.keras.layers.Conv2D(256, (3,3), padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(p2b)
# p2a = tf.keras.layers.BatchNormalization()(p2b)
# p2a = tf.keras.layers.Activation("relu")(p2b)

# cat_512 = tf.keras.layers.Concatenate()([p2a, p2b])

# x = tf.keras.layers.MaxPool2D((2,2), strides=2)(cat_512)

# x = tf.keras.layers.Conv2D(512, (1,1),  padding="same", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Activation("relu")(x)

# x = tf.keras.layers.MaxPool2D((2,2), strides=2)(x)

# x = tf.keras.layers.Conv2D(512, (3,3), kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Activation("relu")(x)

# x = tf.keras.layers.MaxPool2D((2,2), strides=2)(x)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)

# x = tf.keras.layers.Flatten()(x)

# x = tf.keras.layers.Dense(1024, activation="relu", kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=L1_WEIGHT_DECAY, l2=L2_WEIGHT_DECAY))(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(512, activation="relu")(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(5, activation="softmax")(x)

# model = Model(input_layer, x, name='xNet_reg')
# model.summary()

In [5]:
# model=tf.keras.applications.ResNet50V2(input_shape=(i_s,i_s,3),weights=None ,classes=5, classifier_activation="softmax")
# model.summary()

In [6]:
# model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [7]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.0001), loss='categorical_crossentropy', metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [8]:
def convert_to_grayscale(img):
      return tf.image.rgb_to_grayscale(img, name=None)
# the preprocess function assumes 1 argument, the image, you do not need to add that inline
# validation_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=convert_to_grayscale,)

TRAINING_DIR = "emotions_5/train/"
train_datagen = ImageDataGenerator(
      rescale=1./255,
      horizontal_flip=True)
      # rotation_range=40,
      # width_shift_range=0.1,
      # height_shift_range=0.1,
      # shear_range=0.2,
      # zoom_range=0.1,
      # fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, target_size=(i_s,i_s), 
batch_size=64, class_mode="categorical")

VALIDATION_DIR = "emotions_5/validation/"
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, target_size=(i_s,i_s),batch_size=64, class_mode="categorical")



Found 75831 images belonging to 5 classes.
Found 10587 images belonging to 5 classes.


In [9]:
check_points = "checkpoint/checkpoint.hb/"
check_point_dir = os.path.dirname(check_points)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=check_point_dir, verbose=1, monitor="val_acc", save_best_only=True)

In [10]:
## Load checkpoint model
# model = tf.keras.models.load_model("checkpoint/checkpoint.hb/")

In [11]:
history = model.fit(train_generator,
                            epochs=200,
                            verbose=1,  
                            validation_data=validation_generator,
                            callbacks=[cp_callback]) 

Epoch 1/200
1185/1185 [==============================] - ETA: 0s - loss: 17.7180 - acc: 0.4445 - precision: 0.5970 - recall: 0.1704
Epoch 00001: val_acc improved from -inf to 0.46548, saving model to checkpoint/checkpoint.hb
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: checkpoint/checkpoint.hb\assets
1185/1185 [==============================] - 96s 81ms/step - loss: 17.7180 - acc: 0.4445 - precision: 0.5970 - recall: 0.1704 - val_loss: 2.3727 - val_acc: 0.4655 - val_precision: 0.5315 - val_recall: 0.2760
Epoch 2/200
1185/1185 [==============================] - ETA: 0s - loss: 1.5358 - acc: 0.5706 - precision: 0.6798 - recall: 0.3936
Epoch 00002: val_acc improved from 0.46548 to 0.56635, saving model to checkpoint/checkpoint.hb
INFO:tensorflow:Assets written to: checkpoint/checkpoint.hb\assets
1185/1185 [==============================] - 94s 80ms/step - loss: 1.5358 - acc: 0.5706 - precision: 0.6798 - recall: 0.3936 

KeyboardInterrupt: 

In [26]:
faceNet_dir = "faceNet\\xNet_v3.2.0_SGD_80x80_\\"

In [27]:
# Only Run this code if you want to overwrite the old model
tf.saved_model.save(model, faceNet_dir)
# model.save("emotion_model.0_3cnn_2dns_input64x64.model")

INFO:tensorflow:Assets written to: faceNet\xNet_v3.2.0_SGD_80x80_8070\assets


In [ ]:
## Load model
# model = tf.keras.models.load_model(faceNet_dir)

In [28]:
test_dir = "emotions_5/test/"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(i_s,i_s), 
batch_size=64, class_mode="categorical")

model.evaluate(test_generator)

Found 1812 images belonging to 5 classes.
29/29 [==============================] - 1s 50ms/step - loss: 0.9566 - acc: 0.8035 - precision: 0.8285 - recall: 0.7837


[0.9566, 0.8035, 0.8285, 0.7837]

In [ ]:
# root_dir = "img_test"
root_dir = "emotions_5_test\\test\\anger_disgust"
dir_ = os.listdir(root_dir)

test_img_paths = [os.path.join(root_dir,x) for x in dir_]
rows = len(dir_)

fig = plt.gcf()
fig.set_size_inches(16,48)
for i, img in enumerate(test_img_paths):
    img_load_for_test = image.load_img(img, target_size=(i_s,i_s))
    img_test = image.img_to_array(img_load_for_test)
    img_test = np.expand_dims(img_test, axis=0)
    title = model.predict(img_test)

    img_plot = mpimg.imread(img)
    sb = plt.subplot(20,2,i+1)
    sb.set_title(str(title))
    sb.axis("off")
    img_plot = mpimg.imread(img)
    plt.imshow(img_plot)
plt.show()


In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


In [ ]:

import numpy as np
import random
from   tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]

#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Let's prepare a random input image of a cat or dog from the training set.
abc = "emotions/test/anger"
abc_li = os.listdir(abc)
xyz = [os.path.join(abc, f) for f in abc_li]


img_path = random.choice(xyz)
img = load_img(img_path, target_size=(i_s, i_s))  # this is a PIL image

x   = img_to_array(img)                           # Numpy array with shape (150, 150, 3)
x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# -----------------------------------------------------------------------
# Now let's display our representations
# -----------------------------------------------------------------------
fig = plt.gcf()
fig.set_size_inches(16,48)
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------

    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis') 